# Анализ взаимодействия пользователей с карточками Яндекс.Дзен

Почти всё время работы аналитика занимает анализ пользовательского взаимодействия с карточками статей. Необходимо автоматизировать процесс и подготовить дашборд.

**Цель работы** - построение дашборда и подготовка ответов на следующие вопросы:
- сколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек;
- как много карточек генерируют источники с разными темами;
- как соотносятся темы карточек и темы источников.

In [1]:
# импорт библиотек
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

In [2]:
# получение данных из sql-запроса

query = '''
           SELECT *
           FROM dash_visits
        '''

dash_visits = pd.io.sql.read_sql(query, con=engine)

## Обзор данных

In [3]:
# вывод основной информации о датафрейме, подсчет дубликатов и пропусков
def df_info(data: pd.DataFrame):
    print('Первые 10 строк датафрейма')
    display(data.head(10))
    print('Основная информация о датафрейме')
    display(data.info())
    print('Описание данных')
    display(data.describe())
    print('Количество строк-дубликатов')
    display(data.duplicated().sum())
    print('Количество пропусков')
    display(data.isna().sum())

df_info(dash_visits)

Первые 10 строк датафрейма


,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27
5,1040602,Деньги,Авто,18-25,2019-09-24 18:57:00,22
6,1040603,Деньги,Авто,18-25,2019-09-24 18:58:00,24
7,1040604,Деньги,Авто,18-25,2019-09-24 18:59:00,20
8,1040605,Деньги,Авто,18-25,2019-09-24 19:00:00,5
9,1040606,Деньги,Авто,26-30,2019-09-24 18:29:00,1


Основная информация о датафрейме
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


None

Описание данных


,record_id,visits
count,3.074500e+04,30745.000000
mean,1.055969e+06,10.089673
std,8.875461e+03,19.727601
min,1.040597e+06,1.000000
25%,1.048283e+06,1.000000
50%,1.055969e+06,3.000000
75%,1.063655e+06,10.000000
max,1.071341e+06,371.000000


Количество строк-дубликатов


0

Количество пропусков


record_id       0
item_topic      0
source_topic    0
age_segment     0
dt              0
visits          0
dtype: int64

Посмотрим на распределение значений в столбцах с темами карточек, темами источников и возрастными категориями.

In [4]:
# подсчет значений в столбце с темами карточек
dash_visits['item_topic'].value_counts()

Отношения             1536
Интересные факты      1535
Наука                 1505
Подборки              1456
Полезные советы       1424
Общество              1422
Россия                1385
История               1363
Семья                 1287
Путешествия           1247
Деньги                1234
Женщины               1230
Дети                  1229
Туризм                1206
Здоровье              1203
Красота               1193
Культура              1160
Юмор                  1129
Искусство             1119
Рассказы              1109
Психология            1056
Скандалы              1023
Знаменитости           976
Женская психология     914
Шоу                    804
Name: item_topic, dtype: int64

In [5]:
# подсчет значений в столбце с темами источников
dash_visits['source_topic'].value_counts()

Семейные отношения    1822
Россия                1687
Знаменитости          1650
Полезные советы       1578
Путешествия           1563
Кино                  1505
Дети                  1459
История               1437
Семья                 1405
Одежда                1379
Здоровье              1243
Искусство             1228
Авто                  1077
Психология            1055
Сад и дача            1036
Политика              1024
Спорт                 1007
Сделай сам             995
Ремонт                 985
Деньги                 973
Еда                    912
Интерьеры              809
Строительство          758
Музыка                 750
Технологии             741
Финансы                667
Name: source_topic, dtype: int64

In [6]:
# подсчет значений в столбце с возрастными категориями
dash_visits['age_segment'].value_counts()

18-25    7056
26-30    5875
31-35    5552
36-40    5105
41-45    3903
45+      3254
Name: age_segment, dtype: int64

Определим период времени, за который собраны данные.

In [7]:
# определение минимальной и максимальной даты
print('Рассматриваемый период: с', dash_visits['dt'].min(), 'до', dash_visits['dt'].max())

Рассматриваемый период: с 2019-09-24 18:28:00 до 2019-09-24 19:00:00


На основании полученной информации можно сделать следующие выводы:
- типы данных столбцов определены корректно;
- явные дубликаты и пропущенные значения отсутствуют;
- в датасете собрана информация за 24 сентября 2019 (с 18:28 по 19:00).

С данными все в порядке, таким образом, можно приступать к подготовке дашборда.

In [8]:
# преобразование в CSV-файл
dash_visits.to_csv('E:\My Documents\Studying\Data_Analyst\Projects\pr_12_dashboard\dash_visits.csv', index=False)

## Построение дашборда

Ссылка на дашборд на сайте Tableau Public: https://public.tableau.com/app/profile/ekaterina.esipova/viz/Project12-Yandex_Dzen/Yandex_Dzen?publish=yes

## Презентация

Презентация: https://disk.yandex.ru/i/gzpxQCjHrRV5rg

## Общий вывод

В ходе обзора данных неверные типы данных столбцов, явные дубликаты и пропущенные значения обнаружены не были. 

После построения дашборда и более детального анализа можно сделать ряд выводов:

- самое высокое количество событий было зафиксировано в 18:58; 
- больше всего взаимодействий пользователей с карточками происходит по темам: "Наука" (пиковое значение – 4 372 события (7,1%)), "Отношения" (4 145 событий (6,7%)), "Интересные факты" (3 910 событий (6,4%)), "Общество" (3 897 событий (6,4%)) и "Подборки" (3 520 событий (5,7%));
- среди источников наиболее популярными являются "Семейные отношения" (10,7%), "Россия" (9,6%), "Полезные советы" (8,8%), "Путешествия" (7,8%) и "Знаменитости" (7,7%);
- по количеству событий лидируют следующие соотношения тем источников и карточек: "Путешествия" - "Рассказы" (4 587 событий), "Россия" - "Общество" (3 471 событие), "Кино" - "Наука" (3 279 событий), "Россия" - "Россия" (2 847 событий), "Полезные советы" - "Подборки" (2 795 событий). 